In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install turicreate
import turicreate as tc

     |████████████████████████████████| 91.9 MB 2.9 kB/s  eta 0:00:01��█████████    | 80.2 MB 25.0 MB/s eta 0:00:01
     |████████████████████████████████| 86.3 MB 96 kB/s s eta 0:00:01
     |████████████████████████████████| 322 kB 39.5 MB/s eta 0:00:01
     |████████████████████████████████| 3.5 MB 37.8 MB/s eta 0:00:01
     |████████████████████████████████| 449 kB 35.2 MB/s eta 0:00:01
     |████████████████████████████████| 50 kB 4.4 MB/s  eta 0:00:01
     |████████████████████████████████| 3.8 MB 40.8 MB/s eta 0:00:01
  Created wheel for resampy: filename=resampy-0.2.1-py3-none-any.whl size=320848 sha256=b99bf52ce7c67c6653727c7d6e235986512099f5efdbbe420517aef6cef1205c
  Stored in directory: /root/.cache/pip/wheels/71/74/53/d5ceb7c5ee7a168c7d106041863e71ac3273f4a4677743a284
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7539 sha256=93fc708d52b2c78355556a28e7a1b825acfdf29f66ae60fff5f00c3c33da1f47
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a80

      Successfully uninstalled tensorboard-2.3.0
  Attempting uninstall: gast
    Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0
  Attempting uninstall: resampy
    Found existing installation: resampy 0.2.2
    Uninstalling resampy-0.2.2:
      Successfully uninstalled resampy-0.2.2
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

tensorflow-probability 0.11.0 requires gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
librosa 0.8.0 requires resampy>=0.2.2, but you'll have resampy 0.2.1 which is incompat

In [2]:
song_data = tc.SFrame('../input/song-data/song_data.sframe')

In [3]:
song_data.head()

user_id,song_id,listen_count,title,artist
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOAKIMP12A8C130995,1,The Cove,Jack Johnson
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Paco De Lucia
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBXHDL12A81C204C0,1,Stronger,Kanye West
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBYHAJ12A6701BF1D,1,Constellations,Jack Johnson
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODACBL12A8C13C273,1,Learn To Fly,Foo Fighters
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODDNQT12A6D4F5F7E,5,Apuesta Por El Rock 'N'Roll ...,Héroes del Silencio
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODXRTY12AB0180F3B,1,Paper Gangsta,Lady GaGa
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOFGUAY12AB017B0A8,1,Stacked Actors,Foo Fighters
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOFRQTD12A81C233C0,1,Sehr kosmisch,Harmonia
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOHQWYZ12A6D4FA701,1,Heaven's gonna burn youreyes ...,Thievery Corporationfeat. Emiliana Torrini ...


In [6]:
tc.visualization.set_target(target='auto')


In [7]:
song_data['song'].show()


Materializing SArray

In [8]:
len(song_data)


1116609

In [9]:
users = song_data['user_id'].unique()


In [10]:
len(users)


66346

In [11]:
train_data,test_data = song_data.random_split(.8,seed=0)


In [13]:
popularity_model = tc.popularity_recommender.create(train_data,
                                                         user_id='user_id',
                                                         item_id='song')

Warning: Ignoring columns song_id, listen_count, title, artist;

To use one of these as a rating column, specify the column name to be used as target

and use a method that allows the use of a target.

Preparing data set.

Data has 893580 observations with 66085 users and 9952 items.

Data prepared in: 0.984017s

893580 observations to process; with 9952 unique items.

In [14]:
popularity_model.recommend(users=[users[0]])


user_id,song,score,rank
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Sehr kosmisch - Harmonia,4754.0,1
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Undo - Björk,4227.0,2
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,You're The One - DwightYoakam ...,3781.0,3
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Dog Days Are Over (RadioEdit) - Florence + The ...,3633.0,4
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Revelry - Kings Of Leon,3527.0,5
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Horn Concerto No. 4 in Eflat K495: II. Romance ...,3161.0,6
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Secrets - OneRepublic,3148.0,7
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Fireflies - CharttraxxKaraoke ...,2532.0,8
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Tive Sim - Cartola,2521.0,9
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Drop The World - LilWayne / Eminem ...,2053.0,10


In [15]:
popularity_model.recommend(users=[users[1]])


user_id,song,score,rank
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,Sehr kosmisch - Harmonia,4754.0,1
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,Undo - Björk,4227.0,2
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,You're The One - DwightYoakam ...,3781.0,3
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,Dog Days Are Over (RadioEdit) - Florence + The ...,3633.0,4
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,Revelry - Kings Of Leon,3527.0,5
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,Horn Concerto No. 4 in Eflat K495: II. Romance ...,3161.0,6
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,Secrets - OneRepublic,3148.0,7
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,Hey_ Soul Sister - Train,2538.0,8
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,Fireflies - CharttraxxKaraoke ...,2532.0,9
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,Tive Sim - Cartola,2521.0,10


In [16]:
personalized_model = tc.item_similarity_recommender.create(train_data,
                                                                user_id='user_id',
                                                                item_id='song')


Warning: Ignoring columns song_id, listen_count, title, artist;

To use one of these as a rating column, specify the column name to be used as target

and use a method that allows the use of a target.

Preparing data set.

Data has 893580 observations with 66085 users and 9952 items.

...

| 1.49s                               | 100              | 9952            |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 2.5563s

In [17]:
personalized_model.recommend(users=[users[0]])


user_id,song,score,rank
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Cuando Pase El Temblor -Soda Stereo ...,0.019450453611520622,1
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Fireflies - CharttraxxKaraoke ...,0.014473731701190654,2
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Love Is A Losing Game -Amy Winehouse ...,0.014286596041459303,3
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Marry Me - Train,0.014133471709031325,4
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Secrets - OneRepublic,0.013591665488023024,5
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Sehr kosmisch - Harmonia,0.013398789442502536,6
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Te Hacen Falta Vitaminas- Soda Stereo ...,0.01293028317964994,7
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,OMG - Usher featuringwill.i.am ...,0.012777828253232516,8
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Y solo se me ocurreamarte (Unplugged) - ...,0.012341127945826603,9
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,No Dejes Que... -Caifanes ...,0.012104249917543851,10


In [18]:
personalized_model.recommend(users=[users[1]])


user_id,song,score,rank
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,Riot In Cell Block NumberNine - Dr Feelgood ...,0.03749999403953552,1
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,Sei Lá Mangueira -Elizeth Cardoso ...,0.03316326439380646,2
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,The Stallion - Ween,0.032258063554763794,3
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,Rain - Subhumans,0.031415924429893494,4
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,West One (Shine On Me) -The Ruts ...,0.030677199363708496,5
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,Back Against The Wall -Cage The Elephant ...,0.030120477080345154,6
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,Life Less Frightening -Rise Against ...,0.0284431129693985,7
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,A Beggar On A Beach OfGold - Mike And The ...,0.023002490401268005,8
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,Audience Of One - RiseAgainst ...,0.01939384639263153,9
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,Blame It On The Boogie -The Jacksons ...,0.018987342715263367,10


In [19]:
personalized_model.get_similar_items(['With Or Without You - U2'])


song,similar,score,rank
With Or Without You - U2,I Still Haven't FoundWhat I'm Looking For ...,0.04285717010498047,1
With Or Without You - U2,Hold Me_ Thrill Me_ KissMe_ Kill Me - U2 ...,0.033734917640686035,2
With Or Without You - U2,Window In The Skies - U2,0.03283584117889404,3
With Or Without You - U2,Vertigo - U2,0.03007519245147705,4
With Or Without You - U2,Sunday Bloody Sunday - U2,0.02713179588317871,5
With Or Without You - U2,Bad - U2,0.02517986297607422,6
With Or Without You - U2,A Day Without Me - U2,0.023715436458587646,7
With Or Without You - U2,Another Time AnotherPlace - U2 ...,0.020325183868408203,8
With Or Without You - U2,Walk On - U2,0.020202040672302246,9
With Or Without You - U2,Get On Your Boots - U2,0.019685029983520508,10


In [20]:
%matplotlib inline
model_performance = tc.recommender.util.compare_models(test_data,
                                                            [popularity_model,personalized_model],
                                                            user_sample=0.05)

compare_models: using 2931 users to estimate model performance
PROGRESS: Evaluate model M0


recommendations finished on 1000/2931 queries. users per second: 12242.6

recommendations finished on 2000/2931 queries. users per second: 13222.1


Precision and recall summary statistics by cutoff
+--------+----------------------+----------------------+
| cutoff |    mean_precision    |     mean_recall      |
+--------+----------------------+----------------------+
|   1    | 0.023882633913340157 | 0.005885715224025776 |
|   2    | 0.02320027294438758  | 0.011677236571872017 |
|   3    | 0.021380643693847394 | 0.01603285479647693  |
|   4    | 0.02081200955305357  |  0.0212437897540854  |
|   5    |  0.0193790515182532  | 0.024610784600097663 |
|   6    | 0.01859433640395765  | 0.029280087017608866 |
|   7    | 0.017595164985134318 | 0.03242198250702677  |
|   8    | 0.01697372910269534  | 0.03570412877640887  |
|   9    | 0.016376663254861857 | 0.038835444369341623 |
|   10   | 0.015762538382804548 | 0.041796945130228316 |
+--------+----------------------+----------------------+
[10 rows x 3 columns]

PROGRESS: Evaluate model M1


recommendations finished on 1000/2931 queries. users per second: 10660.3

recommendations finished on 2000/2931 queries. users per second: 11543


Precision and recall summary statistics by cutoff
+--------+----------------------+----------------------+
| cutoff |    mean_precision    |     mean_recall      |
+--------+----------------------+----------------------+
|   1    | 0.059024223814397814 | 0.019764647195558142 |
|   2    | 0.04657113613101329  | 0.028238704223862873 |
|   3    |  0.0398043898555669  | 0.03475450068336455  |
|   4    | 0.03488570453770045  | 0.040557978262686555 |
|   5    | 0.032002729443875805 |  0.0454816975927518  |
|   6    | 0.029739565563516427 | 0.05066561390768148  |
|   7    | 0.027196958619681227 | 0.053976194585201705 |
|   8    | 0.025929716820197873 | 0.058361844569623414 |
|   9    | 0.02460290382501232  | 0.06215327187691566  |
|   10   | 0.02313203684749234  | 0.06486076189249174  |
+--------+----------------------+----------------------+
[10 rows x 3 columns]



In [21]:
kayne_west = song_data[song_data['artist'] ==  'Kanye West']['user_id'].unique()
len(kayne_west)

2522

In [22]:
foo_fighters = song_data[song_data['artist'] ==  'Foo Fighters']['user_id'].unique()
len(foo_fighters)

2055

In [25]:
total_count = song_data.groupby(key_column_names='artist', operations={'total_count': tc.aggregate.SUM('listen_count')})


In [26]:
total_count.sort('total_count', ascending=False)


artist,total_count
Kings Of Leon,43218
Dwight Yoakam,40619
Björk,38889
Coldplay,35362
Florence + The Machine,33387
Justin Bieber,29715
Alliance Ethnik,26689
OneRepublic,25754
Train,25402
The Black Keys,22184


In [27]:
total_count.sort('total_count')

artist,total_count
William Tabbert,14
Reel Feelings,24
Beyoncé feat. Bun B andSlim Thug ...,26
Diplo,30
Boggle Karaoke,30
harvey summers,31
Nâdiya,36
Kanye West / Talib Kweli/ Q-Tip / Common / ...,38
Aneta Langerova,38
Jody Bernal,38


In [28]:
train_data,test_data = song_data.random_split(.8,seed=0)


In [29]:
personalized_model = tc.item_similarity_recommender.create(train_data,
                                                                user_id='user_id',
                                                                item_id='song')

Warning: Ignoring columns song_id, listen_count, title, artist;

To use one of these as a rating column, specify the column name to be used as target

and use a method that allows the use of a target.

Preparing data set.

Data has 893580 observations with 66085 users and 9952 items.

...

| 1.24s                               | 100              | 9952            |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 1.30315s

In [30]:
subset_test_users = test_data['user_id'].unique()[0:10000]


In [31]:
subset_test_model = personalized_model.recommend(subset_test_users,k=1)


recommendations finished on 1000/10000 queries. users per second: 12746.5

recommendations finished on 2000/10000 queries. users per second: 14467.1

recommendations finished on 3000/10000 queries. users per second: 15361.1

recommendations finished on 4000/10000 queries. users per second: 15804.1

recommendations finished on 5000/10000 queries. users per second: 16030

recommendations finished on 6000/10000 queries. users per second: 16270.1

recommendations finished on 7000/10000 queries. users per second: 16409.3

recommendations finished on 8000/10000 queries. users per second: 16537.8

recommendations finished on 9000/10000 queries. users per second: 16654.2

recommendations finished on 10000/10000 queries. users per second: 16630.2

In [33]:
song_count = subset_test_model.groupby(key_column_names='song', operations={'count': tc.aggregate.COUNT()})


In [34]:
song_count.sort('count', ascending=False)


song,count
Undo - Björk,436
Secrets - OneRepublic,385
Revelry - Kings Of Leon,224
You're The One - DwightYoakam ...,163
Fireflies - CharttraxxKaraoke ...,120
Sehr kosmisch - Harmonia,95
Horn Concerto No. 4 in Eflat K495: II. Romance ...,93
Hey_ Soul Sister - Train,90
OMG - Usher featuringwill.i.am ...,63
Dog Days Are Over (RadioEdit) - Florence + The ...,47
